In [24]:
import torch

from torchvision import transforms
from torchvision.datasets import UCF101

In [33]:
ucf_data_dir = "datasets\\UFC101\\UCF101\\UCF-101"
ucf_label_dir = "datasets\\UFC101\\ucfTrainTestlist"
frames_per_clip = 4
step_between_clips = 1
batch_size = 32

In [34]:
tfs = transforms.Compose([
            # TODO: this should be done by a video-level transfrom when PyTorch provides transforms.ToTensor() for video
            # scale in [0, 1] of type float
            transforms.Lambda(lambda x: x / 255.),
            # reshape into (T, C, H, W) for easier convolutions
            transforms.Lambda(lambda x: x.permute(0, 3, 1, 2)),
            # rescale to the most common size
            transforms.Lambda(lambda x: nn.functional.interpolate(x, (240, 320))),
])

In [35]:
def custom_collate(batch):
    filtered_batch = []
    for video, _, label in batch:
        filtered_batch.append((video, label))
    return torch.utils.data.dataloader.default_collate(filtered_batch)


## UCF101 Solved

Problema en windows con las rutas

In [36]:
import os

class UCF1012(UCF101):
    def __init__(self, root, annotation_path, frames_per_clip, step_between_clips=1,
                 frame_rate=None, fold=1, train=True, transform=None,
                 _precomputed_metadata=None, num_workers=1, _video_width=0,
                 _video_height=0, _video_min_dimension=0, _audio_samples=0):
        
        super(UCF1012, self).__init__(root, annotation_path, frames_per_clip, step_between_clips,
                 frame_rate, fold, train, transform,
                 _precomputed_metadata, num_workers, _video_width,
                 _video_height, _video_min_dimension, _audio_samples)
    
    def _select_fold(self, video_list, annotation_path, fold, train):
        name = "train" if train else "test"
        name = "{}list{:02d}.txt".format(name, fold)
        f = os.path.join(annotation_path, name)
        selected_files = []
        with open(f, "r") as fid:
            data = fid.readlines()
            data = [x.strip().split(" ") for x in data]
            data = [os.path.join(self.root, x[0]).replace('/', '\\') for x in data]
            selected_files.extend(data)
        
        print(selected_files[0])
        print(video_list[0])
        
        selected_files = set(selected_files)
        
        indices = [i for i in range(len(video_list)) if video_list[i] in selected_files]
        return indices

## Carga del dataset

In [37]:
# create train loader (allowing batches and other extras)
train_dataset = UCF1012(ucf_data_dir, 
                       ucf_label_dir, 
                       frames_per_clip=frames_per_clip,
                       step_between_clips=step_between_clips,
                       train=True)

train_loader = torch.utils.data.DataLoader(train_dataset, 
                                           batch_size=batch_size,
                                           collate_fn=custom_collate)

100.0%


datasets\UFC101\UCF101\UCF-101\ApplyEyeMakeup\v_ApplyEyeMakeup_g08_c01.avi
datasets\UFC101\UCF101\UCF-101\ApplyEyeMakeup\v_ApplyEyeMakeup_g01_c01.avi


In [38]:
print(f"Total number of train samples: {len(train_dataset)}")
print(f"Total number of train samples: {len(train_loader)}")
print()

Total number of train samples: 1757468
Total number of train samples: 54921



In [40]:
dataiter = iter(train_loader)
images, labels = dataiter.next()
print(type(images))
print(images.shape)
print(labels.shape)

AssertionError: torch.Size([5, 240, 320, 3]) x 4

In [14]:
test_dataset = UCF1012(ucf_data_dir,
                       ucf_label_dir,
                       frames_per_clip=frames_per_clip,
                       step_between_clips=step_between_clips,
                       train=False)

test_loader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size=batch_size,
                                          collate_fn=custom_collate)


100.0%


datasets\UFC101\UCF101\UCF-101\ApplyEyeMakeup\v_ApplyEyeMakeup_g01_c01.avi
datasets\UFC101\UCF101\UCF-101\ApplyEyeMakeup\v_ApplyEyeMakeup_g01_c01.avi


In [32]:
print(f"Total number of train samples: {len(test_dataset)}")
print(f"Total number of train samples: {len(test_loader)}")
print()

NameError: name 'test_dataset' is not defined

In [18]:
dataiter = iter(train_loader)
images, labels = dataiter.next()
print(type(images))
print(images.shape)
print(labels.shape)

AssertionError: torch.Size([6, 240, 320, 3]) x 5

# Conv3D test


In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import math
from functools import partial

In [10]:

class BasicBlock(nn.Module):

    def __init__(self):
        super(BasicBlock, self).__init__()
        self.main = nn.Sequential(
            nn.Conv3d(1, 64, 4, 2, 1),
            nn.BatchNorm3d(64),
            nn.LeakyReLU(0.2),

            nn.Conv3d(64, 128, 4, 2, 1),
            nn.BatchNorm3d(128),
            nn.LeakyReLU(0.2),

            nn.Conv3d(128, 256, 4, 2, 1),
            nn.BatchNorm3d(256),
            nn.LeakyReLU(0.2),

            nn.Conv3d(256, 512, 4, 2, 1),
            nn.BatchNorm3d(512),
            nn.LeakyReLU(0.2),

            nn.Conv3d(512, 1, 4, 2, 0),
            
            nn.Dropout(0.2),
            nn.Linear(self.last_channel, 101),
            nn.Softmax()
        )

    def forward(self, x):
        # x's size: batch_size * 1 * 64 * 64 * 64
        return self.main(x)
    


ModuleNotFoundError: No module named 'functions'

In [16]:
net = BasicBlock()

NameError: name 'BasicBlock' is not defined

## Spiking Neural Net